In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import openpathsampling as paths
storage = paths.storage.Storage("mistis.nc", "r")

In [ ]:
mistis = storage.networks.load(0)

In [ ]:
# this should be true if we restored correctly
for ens in mistis.sampling_transitions[0].ensembles:
    print ens in storage.ensembles[:], repr(ens)


In [ ]:
mistis.hist_args['max_lambda'] = { 'bin_width' : 0.02, 'bin_range' : (-0.3, 0.5) }
mistis.hist_args['pathlength'] = { 'bin_width' : 5, 'bin_range' : (0, 150) }

In [ ]:
scheme = storage.schemes[0]
scheme.move_summary(storage)

In [ ]:
for trans in mistis.transitions.values():
    trans._flux = 1.0

In [ ]:
mistis.rate_matrix(storage, force=True)

In [ ]:
trans = mistis.transitions.values()[0]
trans_hists = trans.histograms['max_lambda']

In [ ]:
for hist in trans_hists:
    cross_prob = trans_hists[hist].reverse_cumulative()
    plt.plot(cross_prob.x, np.log(cross_prob))
plt.plot(trans.tcp.x, np.log(trans.tcp))